In [ ]:
# Load ground truth text from the input PDF using pdfplumber
input_pdf_path = r"E:\Btech_AI\Intern\ocrpro\Phable CAM Final.pdf"
file_ground = r"E:\Btech_AI\Intern\ocrpro\Phable_CAM_Final_extracted.txt"

with open(file_ground, "r", encoding="utf-8") as f:
    ground_truth_text = f.read().strip()


# Display first 1000 characters for verification
ground_truth_text[:1000]


'Proposal Summary December 2021  \nBorrower Name: Terrals Technologies Pvt. Ltd.  \nSector: Chronic Disease Management and Healthcare Services Platform  \nProposed Amount: Up to INR 45,00,00,000  \nTranche 1: INR 10,00,00,000  \nTranche 2: Up to 15% of the equity raise (Series B) with an overall cap of INR 35Cr  \nTenor: 18 months from the date of disbursement of each tranche  \nDebt XIRR: 16.56%; Warrants: 7%; XIRR with warrant upside: 31.2%  \nRepayment Schedule: Quarterly Amortization (5 installments); Moratorium for three months i.e., first principal installment at the end of 6 months. The coupons are to be serviced monthly on the last day of each month.  \n\nFiltering Criteria:  \nEquity Raised till Date more than INR 50Cr, Pre-Equity INR 64Cr, Post-Equity INR 192 Cr.  \nDebt to Equity Raised including proposed debt less than 15 percent, Pre-Equity 15.9 percent, Post-Equity 15.16 percent.  \nDebt to Valuation including proposed debt less than 5 percent, Pre-Equity 6.6 percent.  \n

In [8]:
# Load extracted text from both approaches
file_primary = r"Primary Approach using OCR/Output-Primary Approach/extracted_text-PrimaryApproach.txt"
file_mistral = r"E:\Btech_AI\Intern\ocrpro\LLM Approach\Finalised-Mistral-outputs\text_output-mistral.txt"

with open(file_primary, "r", encoding="utf-8") as f:
    text_primary = f.read().strip()

with open(file_mistral, "r", encoding="utf-8") as f:
    text_mistral = f.read().strip()


In [10]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
# Function to compute WER and CER in chunks
def chunked_error_rate(reference, hypothesis, chunk_size=500):
    ref_words = reference.split()
    hyp_words = hypothesis.split()

    ref_chunks = [ref_words[i: i + chunk_size] for i in range(0, len(ref_words), chunk_size)]
    hyp_chunks = [hyp_words[i: i + chunk_size] for i in range(0, len(hyp_words), chunk_size)]

    wer_scores, cer_scores = [], []

    for r_chunk, h_chunk in zip(ref_chunks, hyp_chunks):
        wer_scores.append(len(set(r_chunk) - set(h_chunk)) / len(r_chunk))  # WER Approximation
        cer_scores.append(len(set("".join(r_chunk)) - set("".join(h_chunk))) / len("".join(r_chunk)))  # CER Approximation

    return np.mean(wer_scores), np.mean(cer_scores)

# Compute chunked WER and CER
wer_primary, cer_primary = chunked_error_rate(ground_truth_text, text_primary)
wer_mistral, cer_mistral = chunked_error_rate(ground_truth_text, text_mistral)

# Compute BLEU Score in chunks
def chunked_bleu(reference, hypothesis, chunk_size=500):
    ref_words = reference.split()
    hyp_words = hypothesis.split()

    ref_chunks = [ref_words[i: i + chunk_size] for i in range(0, len(ref_words), chunk_size)]
    hyp_chunks = [hyp_words[i: i + chunk_size] for i in range(0, len(hyp_words), chunk_size)]

    bleu_scores = [sentence_bleu([r_chunk], h_chunk) for r_chunk, h_chunk in zip(ref_chunks, hyp_chunks)]
    
    return np.mean(bleu_scores)

bleu_primary = chunked_bleu(ground_truth_text, text_primary)
bleu_mistral = chunked_bleu(ground_truth_text, text_mistral)

# Output results
{
    "Primary OCR Approach": {
        "WER": wer_primary,
        "CER": cer_primary,
        "BLEU Score": bleu_primary
    },
    "Mistral AI OCR Approach": {
        "WER": wer_mistral,
        "CER": cer_mistral,
        "BLEU Score": bleu_mistral
    }
}


{'Primary OCR Approach': {'WER': 0.4335989010989011,
  'CER': 0.0002393824881465662,
  'BLEU Score': 0.19176308831890435},
 'Mistral AI OCR Approach': {'WER': 0.455514652014652,
  'CER': 0.000635141871039722,
  'BLEU Score': 0.15085038483938618}}